layernormの実装、擬似的バッチサイズ拡張、消失勾配を起こさないようにする、　GANによる画像生成https://qiita.com/keiji_dl/items/45a5775a361151f9189d
https://pytorch.org/tutorials/beginner/dcgan_faces_tutorial.html
generalcomv2dだとだめ？layerNormのためにshapeを使いたい。

In [10]:
!git init
!git config --global user.email "gwsgsgdas@gmail.com"
!git config --global user.name "hayathine"
!git remote add origin https://github.com/hayathine/dl_lecture_competition_pub.git
# !git pull origin event-camera-competition
!git pull origin develop

Reinitialized existing Git repository in /kaggle/working/.git/
fatal: remote origin already exists.
remote: Enumerating objects: 362, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 362 (delta 163), reused 161 (delta 160), pack-reused 195
Receiving objects: 100% (362/362), 67.88 KiB | 7.54 MiB/s, done.
Resolving deltas: 100% (177/177), completed with 4 local objects.
From https://github.com/hayathine/dl_lecture_competition_pub
 * branch            develop    -> FETCH_HEAD
   fe61f92..e1fd3f1  develop    -> origin/develop
Updating fe61f92..e1fd3f1
Fast-forward
 .gitignore                   |   3 +-
 configs/base.yaml            |   9 +++-
 image/memo/1720654695562.png | Bin 0 -> 25860 bytes
 main.py                      |  90 +++++++++++++++++++++++++-----------
 memo.md                      |  32 +++++++++++++
 notebooks/main.ipynb         |   2 +-
 src/datasets.py              |  33 +++++++++----
 src/models/base.py   

In [11]:
!pip install hydra-core　 --upgrade -q
!pip install hdf5plugin --upgrade -q
!pip install timm -q
print('finished')

finished


In [12]:
!ln -s /kaggle/input/dl-last-train/test/test /kaggle/working/data/test
!ln -s /kaggle/input/dl-last-train/train-001/train /kaggle/working/data/train
!ls data

ln: failed to create symbolic link '/kaggle/working/data/test/test': Read-only file system
ln: failed to create symbolic link '/kaggle/working/data/train/train': Read-only file system
test  train


In [ ]:
%env HYDRA_FULL_ERROR=1
load_name = '/kaggle/input/dl_last_model/pytorch/202406242354_1_evflow/8/202407141421_9_bins4drop04batch65.pkl'
save_name = 'bins4drop04batch65'
epochs = 40
dropout = 0.4
shuffle = False
detail = False
num_bins =4
batch_size = 5
batch_extend = 13
sequenceRecurrent=False
learning_rate = 0.001
!python main.py batch_extend=$batch_extend train.num_bins=$num_bins sequenceRecurrent=$sequenceRecurrent save_name=$save_name load_name=$load_name train.epochs=$epochs train.dropout=$dropout data_loader.train.shuffle=$shuffle detail=$detail train.initial_learning_rate=$learning_rate data_loader.train.batch_size=$batch_size

env: HYDRA_FULL_ERROR=1
device: cuda
train data: 2015, test data: 97
-----Model loaded from /kaggle/input/dl_last_model/pytorch/202406242354_1_evflow/8/202407141421_9_bins4drop04batch65.pkl-----
learning_late: 5e-05
100%|█████████████████████████████████████████| 403/403 [09:41<00:00,  1.44s/it]
epoch:0 batch 402 loss: 0.801102562466877
final_train_loss: 0.4661397613278318
Model saved to checkpoints/202407151109_0_bins4drop04batch65
learning_late: 0.0002875
100%|█████████████████████████████████████████| 403/403 [09:18<00:00,  1.39s/it]
epoch:1 batch 402 loss: 0.8005293066096713
final_train_loss: 0.4701944726766026
Model saved to checkpoints/202407151109_1_bins4drop04batch65
learning_late: 0.000525
100%|█████████████████████████████████████████| 403/403 [09:25<00:00,  1.40s/it]
epoch:2 batch 402 loss: 0.8003873887244718
final_train_loss: 0.4782123998963609
Model saved to checkpoints/202407151109_2_bins4drop04batch65
learning_late: 0.0007625
100%|████████████████████████████████████████

In [ ]:
# 予測用
%env HYDRA_FULL_ERROR=1
# dir_name = '/kaggle/input/dl_last_model/pytorch/202406242354_1_evflow/6/'
dir_name = 'checkpoints/'
file_name = '202407132050_12_bins4_batch88.pkl'
load_name = dir_name+file_name
save_name = 'drop04'
epochs = 1
dropout = 0.0
shuffle = False
detail = False
num_bins =4
batch_size = 5
batch_extend = 13
sequenceRecurrent=False
learning_rate = 0.001
!python main.py batch_extend=$batch_extend train.num_bins=$num_bins sequenceRecurrent=$sequenceRecurrent save_name=$save_name load_name=$load_name train.epochs=$epochs train.dropout=$dropout data_loader.train.shuffle=$shuffle detail=$detail train.initial_learning_rate=$learning_rate data_loader.train.batch_size=$batch_size

In [ ]:
# checkpoints/pthをダウンロード
from IPython.display import FileLinks 
FileLinks("checkpoints")
# Dl_last_modelに加える

In [ ]:
# # submission
from IPython.display import FileLink
FileLink("submission.npy")

In [ ]:
!pip install kagglehub

In [ ]:
import kagglehub
from kagglehub.config import get_kaggle_credentials
kagglehub.login()

# Replace with path to directory containing model files.
LOCAL_MODEL_DIR = 'path/to/files'

kagglehub.model_upload(
  handle = 'fewfwfwas/dl_last_model/pyTorch/202406242354_1_evflow',
  local_model_dir = LOCAL_MODEL_DIR,
  version_notes = 'Update 2024-07-14')

# hydraの使い方

https://zenn.dev/gesonanko/articles/417d43669cf2af

### gettingstarted

https://hydra.cc/docs/intro/

#### 2024年6月25日12:19

lossが増えている→過学習？

dropoutを実装する→lossが増加する

#### 2024年6月25日13:28

trainデータの中身を確認する

#### 2024年6月27日

バッチの擬似的拡張
https://kozodoi.me/blog/20210219/gradient-accumulation

    for x, _ in train_dl:
        step_count += 1
        model.train()
        x = x.to(device)

    rec_img, mask = model(x)
        train_loss = torch.mean((rec_img - x) ** 2 * mask) / config["mask_ratio"]
        train_loss.backward()

    if step_count % 8 == 0:  # 8イテレーションごとに更新することで，擬似的にバッチサイズを大きくしている
            optimizer.step()
            optimizer.zero_grad()

    total_train_loss += train_loss.item()

多分勾配消失を起こしているので修正する。
lossをバッチサイズで割っていないことが問題だった。

- 異なるスケールでのロスを足し合わせる．
  - ベースラインモデルはUNet構造なので，デコーダーの中間層の出力は最終的な出力サイズの0.5,0.25,...倍になっています．各中間層の出力を用いてロスを計算することで，勾配消失を防ぎ，性能向上が見込めます．
  →F.interpolateでアップサイズ出来そう

dataのpickle化によるロード短縮化
https://hyper-pigeon.hatenablog.com/entry/2021/08/04/225814

#### 2024/06/29

勾配クリッピングは活用できる？DL第六回
https://www.toolify.ai/ja/ai-news-jp/%E5%8B%BE%E9%85%8D%E5%8B%BE%E9%85%8D%E7%88%86%E7%99%BA%E7%9A%84%E5%A2%97%E5%8A%A0%E5%AF%BE%E5%87%A6%E6%B3%95-1122267
softmaxの勾配が小さくなるのを防ぐためで,Scaled Dot-Product
Attentionと呼ばれる.

GANによる画像生成　第10回
画像が疎なため無理と思う
#### 2024/07/08
layernormの実装

・時系列画像の差を学習
・Validの利用